In [1]:
import os
import cv2 as cv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from google.colab.patches import cv2_imshow
from google.colab import drive
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras.layers.experimental.preprocessing import RandomFlip, RandomRotation, RandomZoom
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
import keras.layers.experimental.preprocessing
from keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [2]:
BASE_PATH = './drive/MyDrive/Visione e Percezione/'
DATASET_PATH = BASE_PATH + 'dataset/imgs/'

INPUT_SIZE = 150
BATCH_SIZE = 32

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
dataset = image_dataset_from_directory(DATASET_PATH, 
                                        image_size = (INPUT_SIZE, INPUT_SIZE))

Found 1495 files belonging to 3 classes.


In [6]:
x = []
y = []
for image, label in tqdm(dataset.unbatch().take(-1)):
    img = image.numpy().astype("uint8")
    x.append(img)
    label = label.numpy()
    y.append(label)

x = np.array(x)
y = np.array(y)

1495it [03:50,  6.48it/s]


In [7]:
x_train, x_val_test, y_train, y_val_test = train_test_split(x, y, test_size=0.4)
x_val, x_test, y_val, y_test = train_test_split(x_val_test, y_val_test, test_size=0.5)

In [8]:
model = Sequential()

model.add(Rescaling(1./255, input_shape=(INPUT_SIZE, INPUT_SIZE, 3)))
model.add(RandomFlip("horizontal"))
model.add(RandomRotation(0.1))
model.add(RandomZoom(0.1))        

model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3))

model.compile(loss=SparseCategoricalCrossentropy(from_logits=True), optimizer='adam',  metrics=['accuracy'])

In [9]:
history = model.fit(
    x_train, y_train,
    validation_data = (x_val, y_val),
    epochs=50,
    verbose=1
)

Epoch 1/50
29/29 [==============================] - 55s 64ms/step - loss: 1.5552 - accuracy: 0.3533 - val_loss: 1.0960 - val_accuracy: 0.5418
Epoch 2/50
29/29 [==============================] - 1s 33ms/step - loss: 1.0851 - accuracy: 0.3772 - val_loss: 1.0390 - val_accuracy: 0.3679
Epoch 3/50
29/29 [==============================] - 1s 33ms/step - loss: 0.8558 - accuracy: 0.5779 - val_loss: 1.1818 - val_accuracy: 0.3579
Epoch 4/50
29/29 [==============================] - 1s 33ms/step - loss: 0.6775 - accuracy: 0.6872 - val_loss: 0.8203 - val_accuracy: 0.5151
Epoch 5/50
29/29 [==============================] - 1s 33ms/step - loss: 0.4477 - accuracy: 0.8024 - val_loss: 0.8564 - val_accuracy: 0.5485
Epoch 6/50
29/29 [==============================] - 1s 33ms/step - loss: 0.3076 - accuracy: 0.8739 - val_loss: 1.1852 - val_accuracy: 0.5552
Epoch 7/50
29/29 [==============================] - 1s 34ms/step - loss: 0.2115 - accuracy: 0.9411 - val_loss: 1.2498 - val_accuracy: 0.5753
Epoch 8/50
2

In [102]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
module_wrapper_32 (ModuleWra (None, 150, 150, 3)       0         
_________________________________________________________________
module_wrapper_33 (ModuleWra (None, 150, 150, 3)       0         
_________________________________________________________________
module_wrapper_34 (ModuleWra (None, 150, 150, 3)       0         
_________________________________________________________________
module_wrapper_35 (ModuleWra (None, 150, 150, 3)       0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 150, 150, 32)      896       
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 75, 75, 32)        0         
_________________________________________________________________
dropout_40 (Dropout)         (None, 75, 75, 32)      

In [16]:
 model.save(BASE_PATH + 'model/cnn_model.h5')

In [10]:
predictions = np.argmax(model.predict(x_test), axis=-1)
predictions = predictions.reshape(1,-1)[0]

conf_mat = confusion_matrix(y_test, predictions)
pd.DataFrame(conf_mat)

,0,1,2
0,79,20,0
1,0,90,0
2,0,2,108


In [11]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       1.00      0.80      0.89        99
           1       0.80      1.00      0.89        90
           2       1.00      0.98      0.99       110

    accuracy                           0.93       299
   macro avg       0.93      0.93      0.92       299
weighted avg       0.94      0.93      0.93       299

